In [14]:
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

### Comparing model performance

In [4]:
path_FL = Path('../results/finland')
path_NL = Path('../results/netherlands')

x = pd.read_pickle(path_FL / 'models' / 'res_men_1.pkl')

In [20]:
x

[{'0': {'precision': 0.05287555397076053,
   'recall': 0.8277511961722488,
   'f1-score': 0.09940148431888916,
   'support': 1254},
  '1': {'precision': 0.9954311821816105,
   'recall': 0.7168032412343498,
   'f1-score': 0.833446972044877,
   'support': 65654},
  'accuracy': 0.7188826448257308,
  'macro avg': {'precision': 0.5241533680761855,
   'recall': 0.7722772187032994,
   'f1-score': 0.4664242281818831,
   'support': 66908},
  'weighted avg': {'precision': 0.9777656600052429,
   'recall': 0.7188826448257308,
   'f1-score': 0.819689378907907,
   'support': 66908}},
 {'0': {'precision': 0.03385317611259034,
   'recall': 0.7876106194690266,
   'f1-score': 0.06491611962071481,
   'support': 226},
  '1': {'precision': 0.9971281560368553,
   'recall': 0.7663938195530212,
   'f1-score': 0.8666666666666667,
   'support': 21746},
  'accuracy': 0.7666120517021664,
  'macro avg': {'precision': 0.5154906660747228,
   'recall': 0.7770022195110239,
   'f1-score': 0.4657913931436908,
   'suppor

In [21]:
def pretty_results(filenames, foldersuffix, results_path):
    reslist = []
    for index1, filename in enumerate(filenames):
        _, sex, nback = filename.split('_')
        res = pd.read_pickle(results_path / f'models{foldersuffix}' / f'{filename}.pkl')
        for index2, cr in enumerate(res):
            traintest = ['train', 'test'][index2 % 2]
            reslist.append([int(nback), traintest, sex, cr['0']['precision'], cr['0']['recall'], cr['0']['support'],
                            cr['1']['precision'], cr['1']['recall'], cr['1']['support']])
    res = pd.DataFrame(reslist).set_axis(['nback', 'traintest', 'sex', 'low_precision', 'low_recall', 'low_support',
                                          'ok_precision', 'ok_recall', 'ok_support'], axis=1)
    return res
    
def get_scores(res_df):
    res_df['old_defrate'] = res_df['low_support'] / (res_df['low_support'] + res_df['ok_support'])
    res_df['new_defrate'] = 1 - res_df['ok_precision']
    res_df['missed_dons'] = 1 - res_df['ok_recall']
    res_df['prevented_defs'] = res_df['low_recall']
    res_df['missed_per_prev'] = (res_df['ok_support'] - res_df['ok_recall'] * res_df['ok_support']) / (res_df['low_support'] - (1 - res_df['ok_precision']) * res_df['ok_support'])
    
    res_df['old_def_n'] = res_df['low_support']
    res_df['new_def_n'] = round((1 - res_df['ok_precision']) * res_df['ok_support'])
    res_df['old_don_n'] = res_df['ok_support']
    res_df['new_don_n'] = res_df['ok_recall'] * res_df['ok_support']
    
    return res_df

In [25]:
res_df = pretty_results([f'res_{sex}_{nback}' for sex in ['men','women'] for nback in range(1,6)], '', path_FL)
res_df_hbonly = pretty_results([f'res_{sex}_{nback}' for sex in ['men','women'] for nback in range(1,6)], '_hbonly', path_FL)

In [30]:
res_FL = pd.concat([df.assign(variables=k) for k,df in {'all':res_df, 'hbonly':res_df_hbonly}.items()])

In [31]:
res_FL

,nback,traintest,sex,low_precision,low_recall,low_support,ok_precision,ok_recall,ok_support,variables
0,1,train,men,0.052876,0.827751,1254,0.995431,0.716803,65654,all
1,1,test,men,0.033853,0.787611,226,0.997128,0.766394,21746,all
2,2,train,men,0.054758,0.878236,1043,0.996821,0.715790,55635,all
3,2,test,men,0.035707,0.800905,221,0.997355,0.776311,21369,all
4,3,train,men,0.051490,0.861905,840,0.996515,0.713201,46503,all
5,3,test,men,0.039185,0.827103,214,0.997739,0.790014,20668,all
6,4,train,men,0.051213,0.853198,688,0.996339,0.716531,38364,all
7,4,test,men,0.040748,0.817308,208,0.997584,0.796781,19693,all
8,5,train,men,0.050344,0.858951,553,0.996515,0.713390,31262,all
9,5,test,men,0.040950,0.826531,196,0.997698,0.795240,18529,all
